In [ ]:
import   os
import   sys
import   numpy          as       np
import   ismrmrd        as       mrd
import   ismrmrd.xsd
from     matplotlib     import   pyplot   as plt
import   h5py

## First - read in file with reconstructed image data

In [ ]:
f = h5py.File('reconstructed_data.h5', 'r')

In [ ]:
f.keys()

In [ ]:
f['2021-12-09 20:16:06']

In [ ]:
list(f.keys())

In [ ]:
f[list(f.keys())[0]]

In [ ]:
dataset_keys = f[list(f.keys())[0]].keys()

In [ ]:
list(dataset_keys)[0]

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]].keys()

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]]['attributes']

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]]['header']

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]]['header'][0]['version']

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]]['attributes'][2]

In [ ]:
f[list(f.keys())[0]][list(dataset_keys)[0]]['data'][1]

In [ ]:
f.close()

In [ ]:
dataset_to_convert = 'reconstructed_data.h5'

In [ ]:
f = h5py.File(dataset_to_convert, 'r')

In [ ]:
subdataset_keys = list(f.keys())

In [ ]:
subdataset_keys

In [ ]:
subdatasets = list()

for key in subdataset_keys:
   subdatasets.append(f[key])

In [ ]:
subdatasets

In [ ]:
subdatasets[0].keys()

In [ ]:
list